# Модель выбытия жилищного фонда 

In [1]:
import requests
from cmasf import pandas_sql as pds
import pandas as pd
import sqlalchemy as sa
import re
from os import path, remove
import glob
from bs4 import BeautifulSoup
import datetime as dt
import numpy as np

strDB_Temp=path.join('Temp') # путь к папке с временными файлами
strBaseDB_URL=r'https://www.reformagkh.ru/opendata?gid=2208161&cids=house_management&page={page_num}&pageSize={page_size}'

strExportDataURL=r'https://www.reformagkh.ru/opendata/export/{num}'

## База данных - сбор

<p>Источник данных - https://www.reformagkh.ru/opendata?gid=2208161&cids=house_management&page=1&pageSize=100</p>
<p>Качаем паспорта домов из <b>"Реестра домов"</b> по каждому региону</i> </p>

<div class="alert alert-block alert-info">
<p>Условие попадания дома в расчеты модели: </p>
    <ul>
        <li>ненулевые данные об общей площади жилых помещений <b>(living_rooms_sq > 0)</b></li>
        <li>ненулевое количество жилых помещений <b>(living_rooms_amount > 0)</b></li>
        <li>соотношение этих параметров находилось в интервале между 3 и 1000 <b>(3 < living_rooms_sq/living_rooms_amount < 1000 )</b></li>
    </ul>
</div>

<div class="alert alert-block alert-success">
<p><b>Поля, используемые в расчетах:</b></p>
     <ol>
         <li>Возраст здания <b>(текущий год - commission_year)</b></li>
         <li>Этажность</li>
      </ul>
</div>


In [2]:
re_a_export=re.compile(r'/opendata/export/\d+') # объект для поиска признака ссылки на файл csv
re_p_date=re.compile('Актуальная версия:') # объект для поиска даты актиализации

re_cap=re.compile('\s(по [А-Яа-я -]+)') # объект для поиска части заголовка - "по Алтайскому краю" - для идентификации ссылки на файл

re_date=re.compile('\d{2}\.\d{2}\.\d{4}') # объект для поиска захвата даты актуализации

re_pass_dt=re.compile('p-5 mx-4') # объект для поиска куска с паспортом набора даных

def delete_temp_files(folder=strDB_Temp):
    """Удаление файлов во временой папке - ОСТОРОЖНО!!! УДАЛЯЕТ ВСЕ ФАЙЛЫ В УКАЗАННОЙ ПАПКЕ!!!
    Вызывать для очистки папки временных файлов после обновления базы данных"""
    files = glob.glob(path.join(folder, '*.zip'))
    for f in files:
        remove(f)

def read_html_page(page_num=1, page_size=100):
    
    dom_html=requests.get(strBaseDB_URL.format(page_num=page_num, page_size=page_size))
    dsp=BeautifulSoup(dom_html.text, 'html.parser')

    p_caps=dsp.findAll('p', text=re.compile('Реестр домов по'))
    if not p_caps: 
#         print('not data')
        return None
    lst_data=list()
    for p in p_caps:
        a_exp=p.find_next('div', class_='row').find('a', href=re_a_export)
        p_date=p.find_next('p', text=re_p_date)
        lst_data.append({'name':re_cap.search(p.text).group(0).strip(),
                        'date':re_date.search(p_date.text).group(0), 
                        'link':r'https://www.reformagkh.ru{suff}'.format(suff=a_exp['href'])})
    return pd.DataFrame(lst_data)

def read_db_password():
    dom_html=requests.get(strBaseDB_URL.format(page_num=1, page_size=10))
    dsp=BeautifulSoup(dom_html.text, 'html.parser')

    p_cap=dsp.find('p', text=re.compile('Реестр домов по'))
    
    div_pass=p_cap.find_next('div', class_=re_pass_dt).findAll('div', class_='row')
    lst_pass=list()
    for d in div_pass:
        dd=d.findAll('div')
        lst_pass.append({'name':dd[0].text, 'description':dd[1].text})
    return lst_pass

In [3]:
pdfDBFiles=pd.concat([read_html_page(page_num=page) for page in range(1, 3)])
print(pdfDBFiles)

lst_fileds=read_db_password()

                                     name        date  \
0                      по Алтайскому краю  01.07.2020   
1                     по Амурской области  01.07.2020   
2                по Архангельской области  01.07.2020   
3                 по Астраханской области  01.07.2020   
4                 по Белгородской области  01.07.2020   
..                                    ...         ...   
80                по Чеченской Республике  01.07.2020   
81                по Чувашской Республике  01.07.2020   
82       по Чукотскому автономному округу  01.07.2020   
83  по Ямало-Ненецкому автономному округу  01.07.2020   
84                 по Ярославской области  01.07.2020   

                                             link  
0   https://www.reformagkh.ru/opendata/export/118  
1   https://www.reformagkh.ru/opendata/export/167  
2   https://www.reformagkh.ru/opendata/export/171  
3    https://www.reformagkh.ru/opendata/export/91  
4   https://www.reformagkh.ru/opendata/export/170  
.. 

In [11]:
for k, v in pdfDBFiles.iterrows():
    print('read for ', v['name'], v['link'], end='...')
    r = requests.get(v['link'])
    with open(path.join(strDB_Temp, '{name}.zip'.format(name=v['name'])), 'wb') as fz_tmp:
        fz_tmp.write(r.content)
    print('done')

read for  по Алтайскому краю https://www.reformagkh.ru/opendata/export/118...done
read for  по Амурской области https://www.reformagkh.ru/opendata/export/167...done
read for  по Архангельской области https://www.reformagkh.ru/opendata/export/171...done
read for  по Астраханской области https://www.reformagkh.ru/opendata/export/91...done
read for  по Белгородской области https://www.reformagkh.ru/opendata/export/170...done
read for  по Брянской области https://www.reformagkh.ru/opendata/export/104...done
read for  по Владимирской области https://www.reformagkh.ru/opendata/export/103...done
read for  по Волгоградской области https://www.reformagkh.ru/opendata/export/90...done
read for  по Вологодской области https://www.reformagkh.ru/opendata/export/169...done
read for  по Воронежской области https://www.reformagkh.ru/opendata/export/102...done
read for  по городу Москве https://www.reformagkh.ru/opendata/export/101...done
read for  по городу Санкт-Петербургу https://www.reformagkh.ru/op

## Собираем данные из ZIP-файлов в один датафрейм

In [12]:
lst_excluded_fileds=['region_id', 'area_id', 'city_id', 'street_id', 'shortname_region', 'formalname_region', 'shortname_area', 'formalname_area',
'shortname_city', 'formalname_city', 'shortname_street', 'formalname_street', 'house_number', 'building', 'block', 'letter',
'management_organization_id', 'method_of_forming_overhaul_fund', 'area_land', 'parking_square', 'playground',
'sportsground', 'other_beautification', 'chute_count', 'electrical_entries_count', 'sewerage_cesspools_volume']

In [13]:
lst_fileds=read_db_password()
pdDataPass=pd.DataFrame(lst_fileds)

In [14]:
zip_files=glob.glob(path.join(strDB_Temp, '*.zip'))

lstDTF=list()
for zf in zip_files:
    print('read ', zf, end='...')
    pdf=pd.read_csv(zf, sep=';')
    pdf.replace('Не заполнено', np.nan)
    pdf=pdf[pdf.columns.difference(lst_excluded_fileds)].set_index('houseguid')
    lstDTF.append(pdf)
    print('done for', pdf.shape)
    
pdfAll=pd.concat(lstDTF)
pdfAll

read  Temp/по Республике Тыва.zip...done for (1065, 33)
read  Temp/по Республике Алтай.zip...done for (1043, 33)
read  Temp/по Республике Карелия.zip...done for (15772, 33)
read  Temp/по Республике Ингушетия.zip...done for (373, 33)
read  Temp/по Ханты-Мансийскому автономному округу.zip...done for (14916, 33)
read  Temp/по Московской области.zip...done for (56850, 33)
read  Temp/по Томской области.zip...done for (7458, 33)
read  Temp/по Республике Северная Осетия-Алания.zip...done for (4106, 33)
read  Temp/по Еврейской автономной области.zip...done for (2078, 33)
read  Temp/по Чеченской Республике.zip...done for (2824, 33)
read  Temp/по Сахалинской области.zip...done for (12462, 33)
read  Temp/по Республике Башкортостан.zip...done for (20222, 33)
read  Temp/по Республике Саха.zip...done for (13456, 33)
read  Temp/по Ульяновской области.zip...done for (7816, 33)
read  Temp/по Нижегородской области.zip...done for (32437, 33)
read  Temp/по Вологодской области.zip...

/home/egor/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done for (16872, 33)
read  Temp/по Республике Татарстан.zip...done for (20002, 33)
read  Temp/по Саратовской области.zip...

/home/egor/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,7,8,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done for (16507, 33)
read  Temp/по Республике Бурятия.zip...done for (5755, 33)
read  Temp/по Липецкой области.zip...done for (7189, 33)
read  Temp/по Республике Марий Эл.zip...done for (6548, 33)
read  Temp/по Оренбургской области.zip...done for (13260, 33)
read  Temp/по Орловской области.zip...done for (5344, 33)
read  Temp/по Тюменской области.zip...done for (9493, 33)
read  Temp/по Приморскому краю.zip...done for (13893, 33)
read  Temp/по Псковской области.zip...done for (6386, 33)
read  Temp/по Ленинградской области.zip...done for (18199, 33)
read  Temp/по Хабаровскому краю.zip...done for (10062, 33)
read  Temp/по Ярославской области.zip...done for (12821, 33)
read  Temp/по Ивановской области.zip...done for (10805, 33)
read  Temp/по Костромской области.zip...done for (8797, 33)
read  Temp/по Забайкальскому краю.zip...done for (8075, 33)
read  Temp/по Республике Коми.zip...done for (10849, 33)
read  Temp/по Республике Мордовия.zip...done for (6260, 33)
read  Temp/по Алтайскому краю

,address,area_common_property,area_non_residential,area_residential,area_total,basement_area,built_year,chute_type,cold_water_type,drainage_type,...,house_type,id,is_alarm,living_quarters_count,project_type,quarters_count,sewerage_type,unliving_quarters_count,ventilation_type,wall_material
houseguid,,,,,,,,,,,,,,,,,,,,,
02ca0341-a6db-421c-a0e3-67966c059835,"Респ. Тыва, г. Ак-Довурак, ул. 50 лет ВЛКСМ, д. 1",NaN,NaN,"2675,10","3189,10",NaN,NaN,Не заполнено,Не заполнено,Не заполнено,...,Не заполнено,6561030,Нет,NaN,NaN,0,Не заполнено,NaN,Не заполнено,Не заполнено
820790a4-fcb5-41d5-ae3b-518205b49e10,"Респ. Тыва, г. Ак-Довурак, ул. 50 лет ВЛКСМ, д. 2",NaN,NaN,"2672,50","3189,10",NaN,NaN,Не заполнено,Не заполнено,Не заполнено,...,Не заполнено,6561031,Нет,NaN,NaN,0,Не заполнено,NaN,Не заполнено,Не заполнено
d37388af-ab11-40f0-ad0b-8366e6df7fb5,"Респ. Тыва, г. Ак-Довурак, ул. 50 лет ВЛКСМ, д. 3",NaN,"196,00","2034,00","3607,00",NaN,NaN,Не заполнено,Центральное,Не заполнено,...,Многоквартирный дом,6937925,Нет,NaN,NaN,60,Центральное,NaN,Не заполнено,Кирпич
924bd201-cc5d-496f-a3b0-c2c4f45c3c48,"Респ. Тыва, г. Ак-Довурак, ул. 50 лет ВЛКСМ, д. 4",NaN,"122,80","2034,00","3505,00","0,00",NaN,Не заполнено,Центральное,Не заполнено,...,Многоквартирный дом,6731134,Нет,NaN,00,60,Центральное,NaN,Не заполнено,Кирпич
5bafbe8f-57c0-4db5-9483-cc170babbd68,"Респ. Тыва, г. Ак-Довурак, ул. 50 лет ВЛКСМ, д. 5","3256,00","0,00","3256,00","3256,00","0,00",1984.0,Отсутствует,Центральное,Наружные водостоки,...,Многоквартирный дом,6932808,Нет,60.0,жилой дом,60,Центральное,0.0,Отсутствует,Кирпич
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0aa33ceb-510d-4f20-8b29-62ba67cf914a,"край. Красноярский, р-н. Эвенкийский, п. Тутон...",NaN,NaN,"127,00","127,00",NaN,NaN,Не заполнено,Не заполнено,Не заполнено,...,Многоквартирный дом,6673015,Да,NaN,NaN,0,Не заполнено,NaN,Не заполнено,Не заполнено
4ada1f98-baf0-02b3-4109-2e901233a83b,"край. Красноярский, р-н. Эвенкийский, п. Тутон...",NaN,NaN,"33,00","61,00",NaN,NaN,Не заполнено,Не заполнено,Не заполнено,...,Многоквартирный дом,6673021,Да,NaN,NaN,0,Не заполнено,NaN,Не заполнено,Не заполнено
130f801b-a9d1-4423-bc06-b90789f8db26,"край. Красноярский, р-н. Эвенкийский, п. Тутон...",NaN,NaN,"98,00","98,00",NaN,NaN,Не заполнено,Не заполнено,Не заполнено,...,Многоквартирный дом,6673028,Да,NaN,NaN,0,Не заполнено,NaN,Не заполнено,Не заполнено


In [8]:
coni = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name='oljia.sqlite3'))

pdfSQL = pds.DataFrameDATA(pdfAll)

pdfAll.to_sql('houses', if_exists='replace', con=coni)
pdDataPass.to_sql('data_pass', con=coni)

In [9]:
delete_temp_files()

In [18]:
pdfSQL.loc[pdfSQL['floor_count_max'].between(1, 5) ].groupby(by=['floor_count_max', 'wall_material', 'built_year']).agg( {'area_residential' : np.nansum, 
                                                                                                                                                      'living_quarters_count':np.nanmean } )

# , 'area_residential':'sum', 'living_quarters_count':'np.mean'

area_residential  \
floor_count_max wall_material                           built_year                     
1.0             Блочные                                 1858.0                  81.9   
                                                        1882.0                 127.2   
                                                        1890.0                 189.7   
                                                        1892.0                 135.6   
                                                        1896.0                 479.9   
...                                                                              ...   
5.0             Смешанные                               2017.0               32321.0   
                                                        2018.0                1510.4   
                Шлакобетон (блоки)                      1981.0                2654.5   
                                                        2018.0                3364.0   
                Шлакокерамзитобетонная 1-слойная панель 1973.0                1734.4   

                                                                    living_quarters_count  
floor_count_max wall_material                           built_year                         
1.0             Блочные                                 1858.0                   4.000000  
                                                        1882.0                   5.000000  
                                                        1890.0                   2.000000  
                                                        1892.0                   3.000000  
                                                        1896.0                   6.500000  
...                                                                                   ...  
5.0             Смешанные                               2017.0                 107.285714  
                                                        2018.0                  27.000000  
                Шлакобетон (блоки)                      1981.0                  90.000000  
                                                        2018.0                  75.000000  
                Шлакокерамзитобетонная 1-слойная панель 1973.0                  60.000000  

[4132 rows x 2 columns]

In [17]:
pdfSQL.loc[pdfSQL['wall_material'].str.contains('бетон', case=False) & (pdfSQL['floor_count_max']==1), ('address', 'wall_material', 'living_quarters_count')].sort_values(by='living_quarters_count')

,address,wall_material,living_quarters_count
houseguid,,,
14941dcc-f248-4d5f-9a0f-45d4efee4fc1,"г. Москва, ул. Головачева, д. 2, стр. инв11",Железобетон,23.0
4f86af32-5a9c-4c4e-aa16-8ded016561cb,"г. Санкт-Петербург, пр-кт. Советский (Усть-Сла...",Железобетон,192.0
dbaa5c03-0777-410e-8859-0f348786ccf8,"обл. Московская, г. Реутов, ул. им академика В...",Керамзитобетонная 1-слойная панель,264.0
9b71d047-ba91-41cd-ab28-127edd581131,"обл. Московская, г. Лыткарино, мкр. 4а, д. 7",Железобетонная панель,306.0


In [23]:
pdfSQL[['area_non_residential', 'area_total','basement_area']].applymap(lambda x: float(x.replace(',', '.')))

AttributeError: 'float' object has no attribute 'replace'